In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]
from sklearn.metrics import mean_squared_error
from math import sqrt


In [2]:
train = pd.read_csv('Train (1).csv')
test = pd.read_csv('Test (1).csv')

In [3]:
train.head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822 entries, 0 to 2821
Data columns (total 63 columns):
ward                 2822 non-null object
total_households     2822 non-null float64
total_individuals    2822 non-null float64
target               2822 non-null float64
dw_00                2822 non-null float64
dw_01                2822 non-null float64
dw_02                2822 non-null float64
dw_03                2822 non-null float64
dw_04                2822 non-null float64
dw_05                2822 non-null float64
dw_06                2822 non-null float64
dw_07                2822 non-null float64
dw_08                2822 non-null float64
dw_09                2822 non-null float64
dw_10                2822 non-null float64
dw_11                2822 non-null float64
dw_12                2822 non-null int64
dw_13                2822 non-null int64
psa_00               2822 non-null float64
psa_01               2822 non-null float64
psa_02               2822 non-null flo

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 62 columns):
ward                 1013 non-null object
total_households     1013 non-null float64
total_individuals    1013 non-null float64
dw_00                1013 non-null float64
dw_01                1013 non-null float64
dw_02                1013 non-null float64
dw_03                1013 non-null float64
dw_04                1013 non-null float64
dw_05                1013 non-null float64
dw_06                1013 non-null float64
dw_07                1013 non-null float64
dw_08                1013 non-null float64
dw_09                1013 non-null float64
dw_10                1013 non-null float64
dw_11                1013 non-null float64
dw_12                1013 non-null float64
dw_13                1013 non-null float64
psa_00               1013 non-null float64
psa_01               1013 non-null float64
psa_02               1013 non-null float64
psa_03               1013 non-null

In [6]:
train.shape

(2822, 63)

In [7]:
test.shape

(1013, 62)

In [8]:
train.drop(['ADM4_PCODE', 'ward'], axis = 1, inplace = True)
test.drop(['ADM4_PCODE', 'ward'], axis = 1, inplace = True)


In [9]:
y = train['target']
X = train.drop(['target'], axis=1)

In [10]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import GridSearchCV

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
X_train.columns

Index(['total_households', 'total_individuals', 'dw_00', 'dw_01', 'dw_02',
       'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10',
       'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01', 'psa_02', 'psa_03',
       'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01', 'lln_00', 'lln_01',
       'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06',
       'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_13',
       'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04', 'lgt_00',
       'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05', 'pw_06', 'pw_07',
       'pw_08', 'lat', 'lon', 'NL'],
      dtype='object')

In [76]:
estimator = lgb.LGBMRegressor(learning_rate = 0.15, metric = 'l1', 
                        n_estimators = 24, num_leaves = 38)


param_grid = {
    'n_estimators': [x for x in range(24,40,2)],
    'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
        eval_set = [(X_test, y_test)],
        eval_metric = ['auc', 'binary_logloss'],
        early_stopping_rounds = 5,
              verbose = 1)

[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12507	valid_0's auc: 0.673759	valid_0's l1: 7.17636
[3]	valid_0's binary_logloss: -3.12094	valid_0's auc: 0.662234	valid_0's l1: 6.60381
[4]	valid_0's binary_logloss: -3.11644	valid_0's auc: 0.746454	valid_0's l1: 6.07745
[5]	valid_0's binary_logloss: -3.11156	valid_0's auc: 0.73227	valid_0's l1: 5.62969
[6]	valid_0's binary_logloss: -3.10588	valid_0's auc: 0.703901	valid_0's l1: 5.231
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
[1]	valid_0's binary_logloss: -3.14238	valid_0's auc: 0.470745	valid_0's l1: 7.90232
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13842	valid_0's auc: 0.458333	valid_0's l1: 7.23041
[3]	valid_0's binary_logloss: -3.13376	valid_0's auc: 0.468085	valid_0's l1: 6.65

[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12507	valid_0's auc: 0.673759	valid_0's l1: 7.17636
[3]	valid_0's binary_logloss: -3.12094	valid_0's auc: 0.662234	valid_0's l1: 6.60381
[4]	valid_0's binary_logloss: -3.11644	valid_0's auc: 0.746454	valid_0's l1: 6.07745
[5]	valid_0's binary_logloss: -3.11156	valid_0's auc: 0.73227	valid_0's l1: 5.62969
[6]	valid_0's binary_logloss: -3.10588	valid_0's auc: 0.703901	valid_0's l1: 5.231
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
[1]	valid_0's binary_logloss: -3.14238	valid_0's auc: 0.470745	valid_0's l1: 7.90232
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13842	valid_0's auc: 0.458333	valid_0's l1: 7.23041
[3]	valid_0's binary_logloss: -3.13376	valid_0's auc: 0.468085	valid_0's l1: 6.65

[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12309	valid_0's auc: 0.779255	valid_0's l1: 6.85848
[3]	valid_0's binary_logloss: -3.11718	valid_0's auc: 0.738475	valid_0's l1: 6.17713
[4]	valid_0's binary_logloss: -3.11108	valid_0's auc: 0.704787	valid_0's l1: 5.60881
[5]	valid_0's binary_logloss: -3.10459	valid_0's auc: 0.664007	valid_0's l1: 5.12563
[6]	valid_0's binary_logloss: -3.09879	valid_0's auc: 0.593972	valid_0's l1: 4.74333
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
[1]	valid_0's binary_logloss: -3.14123	valid_0's auc: 0.470745	valid_0's l1: 7.72115
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13523	valid_0's auc: 0.458333	valid_0's l1: 6.9063
[3]	valid_0's binary_logloss: -3.12873	valid_0's auc: 0.542553	valid_0's l1: 6.

[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12309	valid_0's auc: 0.779255	valid_0's l1: 6.85848
[3]	valid_0's binary_logloss: -3.11718	valid_0's auc: 0.738475	valid_0's l1: 6.17713
[4]	valid_0's binary_logloss: -3.11108	valid_0's auc: 0.704787	valid_0's l1: 5.60881
[5]	valid_0's binary_logloss: -3.10459	valid_0's auc: 0.664007	valid_0's l1: 5.12563
[6]	valid_0's binary_logloss: -3.09879	valid_0's auc: 0.593972	valid_0's l1: 4.74333
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
[1]	valid_0's binary_logloss: -3.14123	valid_0's auc: 0.470745	valid_0's l1: 7.72115
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13523	valid_0's auc: 0.458333	valid_0's l1: 6.9063
[3]	valid_0's binary_logloss: -3.12873	valid_0's auc: 0.542553	valid_0's l1: 6.

[1]	valid_0's binary_logloss: -3.12759	valid_0's auc: 0.771277	valid_0's l1: 7.4674
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12042	valid_0's auc: 0.772163	valid_0's l1: 6.54788
[3]	valid_0's binary_logloss: -3.11318	valid_0's auc: 0.715426	valid_0's l1: 5.80147
[4]	valid_0's binary_logloss: -3.10592	valid_0's auc: 0.704787	valid_0's l1: 5.19403
[5]	valid_0's binary_logloss: -3.09735	valid_0's auc: 0.753546	valid_0's l1: 4.68254
[6]	valid_0's binary_logloss: -3.08891	valid_0's auc: 0.721631	valid_0's l1: 4.27049
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12759	valid_0's auc: 0.771277	valid_0's l1: 7.4674
[1]	valid_0's binary_logloss: -3.13999	valid_0's auc: 0.470745	valid_0's l1: 7.54183
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13291	valid_0's auc: 0.501773	valid_0's l1: 6.64119
[3]	valid_0's binary_logloss: -3.12447	valid_0's auc: 0.56383	valid_0's l1: 5.85

...

[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.11786	valid_0's auc: 0.859043	valid_0's l1: 6.24611
[3]	valid_0's binary_logloss: -3.10876	valid_0's auc: 0.750887	valid_0's l1: 5.41736
[4]	valid_0's binary_logloss: -3.1001	valid_0's auc: 0.765071	valid_0's l1: 4.76799
[5]	valid_0's binary_logloss: -3.09037	valid_0's auc: 0.726064	valid_0's l1: 4.2691
[6]	valid_0's binary_logloss: -3.0818	valid_0's auc: 0.792553	valid_0's l1: 3.88162
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
[1]	valid_0's binary_logloss: -3.13866	valid_0's auc: 0.470745	valid_0's l1: 7.3646
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13005	valid_0's auc: 0.507092	valid_0's l1: 6.35636
[3]	valid_0's binary_logloss: -3.11896	valid_0's auc: 0.41844	valid_0's l1: 5.5446

[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.11786	valid_0's auc: 0.859043	valid_0's l1: 6.24611
[3]	valid_0's binary_logloss: -3.10876	valid_0's auc: 0.750887	valid_0's l1: 5.41736
[4]	valid_0's binary_logloss: -3.1001	valid_0's auc: 0.765071	valid_0's l1: 4.76799
[5]	valid_0's binary_logloss: -3.09037	valid_0's auc: 0.726064	valid_0's l1: 4.2691
[6]	valid_0's binary_logloss: -3.0818	valid_0's auc: 0.792553	valid_0's l1: 3.88162
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
[1]	valid_0's binary_logloss: -3.13866	valid_0's auc: 0.470745	valid_0's l1: 7.3646
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13005	valid_0's auc: 0.507092	valid_0's l1: 6.35636
[3]	valid_0's binary_logloss: -3.11896	valid_0's auc: 0.41844	valid_0's l1: 5.5446

[2]	valid_0's binary_logloss: -3.11492	valid_0's auc: 0.859043	valid_0's l1: 5.96061
[3]	valid_0's binary_logloss: -3.10428	valid_0's auc: 0.804078	valid_0's l1: 5.11855
[4]	valid_0's binary_logloss: -3.09341	valid_0's auc: 0.827128	valid_0's l1: 4.46351
[5]	valid_0's binary_logloss: -3.08291	valid_0's auc: 0.835106	valid_0's l1: 3.97934
[6]	valid_0's binary_logloss: -3.07398	valid_0's auc: 0.762411	valid_0's l1: 3.62937
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12537	valid_0's auc: 0.771277	valid_0's l1: 7.09863
[1]	valid_0's binary_logloss: -3.13723	valid_0's auc: 0.470745	valid_0's l1: 7.1894
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12708	valid_0's auc: 0.507092	valid_0's l1: 6.08014
[3]	valid_0's binary_logloss: -3.11416	valid_0's auc: 0.41844	valid_0's l1: 5.2275
[4]	valid_0's binary_logloss: -3.10141	valid_0's auc: 0.427305	valid_0's l1: 4.56728
[5]	valid_0's binary_logloss: -3.08845	valid_0's auc: 0.4

[4]	valid_0's binary_logloss: -3.09341	valid_0's auc: 0.827128	valid_0's l1: 4.46351
[5]	valid_0's binary_logloss: -3.08291	valid_0's auc: 0.835106	valid_0's l1: 3.97934
[6]	valid_0's binary_logloss: -3.07398	valid_0's auc: 0.762411	valid_0's l1: 3.62937
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12537	valid_0's auc: 0.771277	valid_0's l1: 7.09863
[1]	valid_0's binary_logloss: -3.13723	valid_0's auc: 0.470745	valid_0's l1: 7.1894
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12708	valid_0's auc: 0.507092	valid_0's l1: 6.08014
[3]	valid_0's binary_logloss: -3.11416	valid_0's auc: 0.41844	valid_0's l1: 5.2275
[4]	valid_0's binary_logloss: -3.10141	valid_0's auc: 0.427305	valid_0's l1: 4.56728
[5]	valid_0's binary_logloss: -3.08845	valid_0's auc: 0.466312	valid_0's l1: 4.06269
[6]	valid_0's binary_logloss: -3.0787	valid_0's auc: 0.413121	valid_0's l1: 3.66351
Early stopping, best iteration is:
[1]	valid_0's binary_lo

GridSearchCV(cv=None, error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.15, max_depth=-1,
       metric='l1', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=24, n_jobs=-1, num_leaves=38,
       objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=200000,
       subsample_freq=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [24, 26, 28, 30, 32, 34, 36, 38], 'learning_rate': [0.1, 0.125, 0.15, 0.175, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [77]:
print('Best parameters found by grid search are:', gridsearch.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.2, 'n_estimators': 24}


In [78]:
gbm = lgb.LGBMRegressor(learning_rate = 0.2, metric = 'l1', 
                        n_estimators = 24)


gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)

[1]	valid_0's binary_logloss: -3.13059	valid_0's auc: 0.800532	valid_0's l1: 7.19913
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.11882	valid_0's auc: 0.804965	valid_0's l1: 6.08439
[3]	valid_0's binary_logloss: -3.10789	valid_0's auc: 0.680851	valid_0's l1: 5.21651
[4]	valid_0's binary_logloss: -3.09711	valid_0's auc: 0.606383	valid_0's l1: 4.56129
[5]	valid_0's binary_logloss: -3.08637	valid_0's auc: 0.597518	valid_0's l1: 4.06072
[6]	valid_0's binary_logloss: -3.07677	valid_0's auc: 0.560284	valid_0's l1: 3.66195
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.13059	valid_0's auc: 0.800532	valid_0's l1: 7.19913


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.2, max_depth=-1,
       metric='l1', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=24, n_jobs=-1, num_leaves=31,
       objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=200000,
       subsample_freq=0)

In [79]:
y_pred = gbm.predict(test)
#y_predict = y_pred.astype('int64')

In [80]:
gbm.score(X_train, y_train)

0.3102035537692649

In [81]:
print("RMSE:", sqrt(mean_squared_error(y_train, rf.predict(X_train))))

RMSE: 1.3307687792944694


In [28]:
dsn_test_s = pd.read_csv('Test (1).csv')
submission = pd.DataFrame({'ward':dsn_test_s['ward'], 'target': y_pred})
submission

,ward,target
0,21001001: Ward 1,23.574435
1,21001002: Ward 2,22.065092
2,21001003: Ward 3,25.100745
3,21001004: Ward 4,23.106872
4,21001005: Ward 5,24.017683
...,...,...
1008,19100107: Ward 107,21.115486
1009,19100108: Ward 108,24.017683
1010,19100109: Ward 109,23.574435
1011,19100110: Ward 110,22.065092


In [29]:
filename = 'women_submission.csv'
submission.to_csv(filename,index = False)
print('saved file:' + filename)

saved file:women_submission.csv


In [87]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100) # My go-to quick model for regression tasks.
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [88]:
rf.score(X_train, y_train) # Very good!

0.9834585318009083

In [89]:
y_predict = rf.predict(test)

In [84]:
print("RMSE:", sqrt(mean_squared_error(y_train, rf.predict(X_train))))

RMSE: 1.3098719111036685


In [90]:
dsn_test_s = pd.read_csv('Test (1).csv')
submission = pd.DataFrame({'ward':dsn_test_s['ward'], 'target': y_predict})
submission

,ward,target
0,21001001: Ward 1,18.658697
1,21001002: Ward 2,13.947497
2,21001003: Ward 3,21.683328
3,21001004: Ward 4,16.224131
4,21001005: Ward 5,22.556239
...,...,...
1008,19100107: Ward 107,7.829122
1009,19100108: Ward 108,21.670267
1010,19100109: Ward 109,17.530556
1011,19100110: Ward 110,14.442273


In [92]:
filename = 'women_submission_RandomForest.csv'
submission.to_csv(filename,index = False)
print('saved file:' + filename)

saved file:women_submission_RandomForest.csv


In [14]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(iterations= 200, learning_rate= 0.1, depth= 6 ) # My go-to quick model for regression tasks.
cat.fit(X_train, y_train)

0:	learn: 24.3817814	total: 116ms	remaining: 23s
1:	learn: 22.2033787	total: 143ms	remaining: 14.1s
2:	learn: 20.2478816	total: 169ms	remaining: 11.1s
3:	learn: 18.4482455	total: 197ms	remaining: 9.64s
4:	learn: 16.8130609	total: 223ms	remaining: 8.7s
5:	learn: 15.3332273	total: 250ms	remaining: 8.08s
6:	learn: 14.0202521	total: 277ms	remaining: 7.64s
7:	learn: 12.8450237	total: 303ms	remaining: 7.28s
8:	learn: 11.8366519	total: 346ms	remaining: 7.34s
9:	learn: 10.8982692	total: 375ms	remaining: 7.12s
10:	learn: 10.0454226	total: 401ms	remaining: 6.88s
11:	learn: 9.2791521	total: 428ms	remaining: 6.7s
12:	learn: 8.6210730	total: 455ms	remaining: 6.54s
13:	learn: 8.0054043	total: 481ms	remaining: 6.39s
14:	learn: 7.4575184	total: 517ms	remaining: 6.38s
15:	learn: 6.9960442	total: 545ms	remaining: 6.26s
16:	learn: 6.5544453	total: 571ms	remaining: 6.14s
17:	learn: 6.2001340	total: 598ms	remaining: 6.05s
18:	learn: 5.8676906	total: 625ms	remaining: 5.95s
19:	learn: 5.5863508	total: 652ms	

165:	learn: 2.3420741	total: 5.22s	remaining: 1.07s
166:	learn: 2.3373790	total: 5.24s	remaining: 1.04s
167:	learn: 2.3294036	total: 5.27s	remaining: 1s
168:	learn: 2.3229992	total: 5.3s	remaining: 972ms
169:	learn: 2.3148128	total: 5.33s	remaining: 940ms
170:	learn: 2.3092040	total: 5.36s	remaining: 908ms
171:	learn: 2.3040135	total: 5.38s	remaining: 876ms
172:	learn: 2.2978837	total: 5.41s	remaining: 845ms
173:	learn: 2.2925978	total: 5.44s	remaining: 813ms
174:	learn: 2.2845541	total: 5.47s	remaining: 781ms
175:	learn: 2.2744608	total: 5.5s	remaining: 750ms
176:	learn: 2.2676010	total: 5.53s	remaining: 718ms
177:	learn: 2.2617758	total: 5.55s	remaining: 686ms
178:	learn: 2.2544762	total: 5.58s	remaining: 655ms
179:	learn: 2.2464822	total: 5.61s	remaining: 624ms
180:	learn: 2.2428544	total: 5.64s	remaining: 592ms
181:	learn: 2.2349738	total: 5.67s	remaining: 560ms
182:	learn: 2.2276601	total: 5.7s	remaining: 529ms
183:	learn: 2.2192183	total: 5.72s	remaining: 498ms
184:	learn: 2.2160

In [15]:
cat.score(X_train, y_train) # Very good!

0.9573573630447073

In [17]:
y_predict = cat.predict(test)

In [18]:
print("RMSE:", sqrt(mean_squared_error(y_train, cat.predict(X_train))))

RMSE: 2.1203527678082783


In [19]:
dsn_test_s = pd.read_csv('Test (1).csv')
submission = pd.DataFrame({'ward':dsn_test_s['ward'], 'target': y_predict})
submission

,ward,target
0,21001001: Ward 1,17.271489
1,21001002: Ward 2,12.545411
2,21001003: Ward 3,23.389417
3,21001004: Ward 4,16.765958
4,21001005: Ward 5,20.567299
...,...,...
1008,19100107: Ward 107,6.887361
1009,19100108: Ward 108,22.063097
1010,19100109: Ward 109,15.880068
1011,19100110: Ward 110,11.347963


In [20]:
filename = 'women_submission_Catboost.csv'
submission.to_csv(filename,index = False)
print('saved file:' + filename)

saved file:women_submission_Catboost.csv


In [13]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [14]:
 parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.01, 0.05, 0.1],
                  'iterations'    : [100, 200, 600 ]
                 }
grid = GridSearchCV(estimator=cat, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)    
    # Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n",grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n",grid.best_params_)
    
print("\n ========================================================")


0:	learn: 25.4960373	total: 260ms	remaining: 2m 35s
1:	learn: 24.3495485	total: 368ms	remaining: 1m 49s
2:	learn: 23.3020748	total: 480ms	remaining: 1m 35s
3:	learn: 22.2292416	total: 592ms	remaining: 1m 28s
4:	learn: 21.2494560	total: 722ms	remaining: 1m 25s
5:	learn: 20.2911026	total: 824ms	remaining: 1m 21s
6:	learn: 19.3743728	total: 936ms	remaining: 1m 19s
7:	learn: 18.4977440	total: 1.04s	remaining: 1m 16s
8:	learn: 17.6962951	total: 1.15s	remaining: 1m 15s
9:	learn: 16.9269733	total: 1.26s	remaining: 1m 14s
10:	learn: 16.1883545	total: 1.36s	remaining: 1m 13s
11:	learn: 15.4859606	total: 1.47s	remaining: 1m 11s
12:	learn: 14.8305984	total: 1.57s	remaining: 1m 10s
13:	learn: 14.1924513	total: 1.68s	remaining: 1m 10s
14:	learn: 13.5882176	total: 1.79s	remaining: 1m 9s
15:	learn: 13.0196656	total: 1.89s	remaining: 1m 9s
16:	learn: 12.4842285	total: 2s	remaining: 1m 8s
17:	learn: 11.9691591	total: 2.11s	remaining: 1m 8s
18:	learn: 11.4739187	total: 2.21s	remaining: 1m 7s
19:	learn: 

162:	learn: 3.4126204	total: 16.2s	remaining: 43.5s
163:	learn: 3.4126191	total: 16.2s	remaining: 43.1s
164:	learn: 3.4125286	total: 16.2s	remaining: 42.8s
165:	learn: 3.4125187	total: 16.3s	remaining: 42.5s
166:	learn: 3.4124251	total: 16.3s	remaining: 42.2s
167:	learn: 3.4111232	total: 16.4s	remaining: 42.1s
168:	learn: 3.4110209	total: 16.4s	remaining: 41.8s
169:	learn: 3.4064332	total: 16.5s	remaining: 41.8s
170:	learn: 3.4058028	total: 16.6s	remaining: 41.7s
171:	learn: 3.3956101	total: 16.7s	remaining: 41.6s
172:	learn: 3.3860661	total: 16.8s	remaining: 41.5s
173:	learn: 3.3787737	total: 16.9s	remaining: 41.4s
174:	learn: 3.3755531	total: 17s	remaining: 41.3s
175:	learn: 3.3727266	total: 17.1s	remaining: 41.2s
176:	learn: 3.3641764	total: 17.2s	remaining: 41.1s
177:	learn: 3.3638270	total: 17.3s	remaining: 41s
178:	learn: 3.3633774	total: 17.4s	remaining: 40.9s
179:	learn: 3.3633677	total: 17.4s	remaining: 40.6s
180:	learn: 3.3571183	total: 17.5s	remaining: 40.5s
181:	learn: 3.35

321:	learn: 2.8238538	total: 31.4s	remaining: 27.1s
322:	learn: 2.8215959	total: 31.5s	remaining: 27s
323:	learn: 2.8178121	total: 31.6s	remaining: 26.9s
324:	learn: 2.8133447	total: 31.7s	remaining: 26.8s
325:	learn: 2.8116193	total: 31.8s	remaining: 26.7s
326:	learn: 2.8111380	total: 31.9s	remaining: 26.6s
327:	learn: 2.8100727	total: 32s	remaining: 26.5s
328:	learn: 2.8035540	total: 32.1s	remaining: 26.4s
329:	learn: 2.8029638	total: 32.2s	remaining: 26.3s
330:	learn: 2.8018321	total: 32.3s	remaining: 26.2s
331:	learn: 2.8000015	total: 32.4s	remaining: 26.2s
332:	learn: 2.7988697	total: 32.6s	remaining: 26.1s
333:	learn: 2.7978310	total: 32.7s	remaining: 26s
334:	learn: 2.7968334	total: 32.8s	remaining: 25.9s
335:	learn: 2.7876109	total: 32.9s	remaining: 25.9s
336:	learn: 2.7844589	total: 33s	remaining: 25.8s
337:	learn: 2.7805344	total: 33.1s	remaining: 25.7s
338:	learn: 2.7789395	total: 33.2s	remaining: 25.6s
339:	learn: 2.7767207	total: 33.3s	remaining: 25.5s
340:	learn: 2.776000

481:	learn: 2.5211483	total: 47.4s	remaining: 11.6s
482:	learn: 2.5207204	total: 47.5s	remaining: 11.5s
483:	learn: 2.5204480	total: 47.6s	remaining: 11.4s
484:	learn: 2.5129959	total: 47.7s	remaining: 11.3s
485:	learn: 2.5110128	total: 47.8s	remaining: 11.2s
486:	learn: 2.5107505	total: 47.9s	remaining: 11.1s
487:	learn: 2.5104093	total: 48s	remaining: 11s
488:	learn: 2.5096272	total: 48.1s	remaining: 10.9s
489:	learn: 2.5081127	total: 48.2s	remaining: 10.8s
490:	learn: 2.5069916	total: 48.3s	remaining: 10.7s
491:	learn: 2.5068525	total: 48.4s	remaining: 10.6s
492:	learn: 2.5054833	total: 48.5s	remaining: 10.5s
493:	learn: 2.5029067	total: 48.6s	remaining: 10.4s
494:	learn: 2.5017150	total: 48.7s	remaining: 10.3s
495:	learn: 2.4976488	total: 48.8s	remaining: 10.2s
496:	learn: 2.4965473	total: 48.9s	remaining: 10.1s
497:	learn: 2.4949188	total: 49s	remaining: 10s
498:	learn: 2.4926970	total: 49.1s	remaining: 9.94s
499:	learn: 2.4921312	total: 49.2s	remaining: 9.84s
500:	learn: 2.491131

In [15]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(iterations= 600, learning_rate= 0.05, depth= 6 ) # My go-to quick model for regression tasks.
cat.fit(X_train, y_train)

0:	learn: 25.4960373	total: 128ms	remaining: 1m 16s
1:	learn: 24.3495485	total: 234ms	remaining: 1m 10s
2:	learn: 23.3020748	total: 353ms	remaining: 1m 10s
3:	learn: 22.2292416	total: 462ms	remaining: 1m 8s
4:	learn: 21.2494560	total: 571ms	remaining: 1m 8s
5:	learn: 20.2911026	total: 678ms	remaining: 1m 7s
6:	learn: 19.3743728	total: 789ms	remaining: 1m 6s
7:	learn: 18.4977440	total: 896ms	remaining: 1m 6s
8:	learn: 17.6962951	total: 1.01s	remaining: 1m 6s
9:	learn: 16.9269733	total: 1.14s	remaining: 1m 7s
10:	learn: 16.1883545	total: 1.25s	remaining: 1m 7s
11:	learn: 15.4859606	total: 1.36s	remaining: 1m 6s
12:	learn: 14.8305984	total: 1.46s	remaining: 1m 6s
13:	learn: 14.1924513	total: 1.57s	remaining: 1m 5s
14:	learn: 13.5882176	total: 1.69s	remaining: 1m 5s
15:	learn: 13.0196656	total: 1.8s	remaining: 1m 5s
16:	learn: 12.4842285	total: 1.91s	remaining: 1m 5s
17:	learn: 11.9691591	total: 2.02s	remaining: 1m 5s
18:	learn: 11.4739187	total: 2.12s	remaining: 1m 4s
19:	learn: 11.010613

161:	learn: 3.4137207	total: 16.8s	remaining: 45.4s
162:	learn: 3.4126204	total: 16.9s	remaining: 45.3s
163:	learn: 3.4126191	total: 16.9s	remaining: 45s
164:	learn: 3.4125286	total: 16.9s	remaining: 44.6s
165:	learn: 3.4125187	total: 16.9s	remaining: 44.3s
166:	learn: 3.4124251	total: 17s	remaining: 44s
167:	learn: 3.4111232	total: 17.1s	remaining: 43.9s
168:	learn: 3.4110209	total: 17.1s	remaining: 43.6s
169:	learn: 3.4064332	total: 17.2s	remaining: 43.5s
170:	learn: 3.4058028	total: 17.3s	remaining: 43.4s
171:	learn: 3.3956101	total: 17.4s	remaining: 43.3s
172:	learn: 3.3860661	total: 17.5s	remaining: 43.3s
173:	learn: 3.3787737	total: 17.6s	remaining: 43.2s
174:	learn: 3.3755531	total: 17.7s	remaining: 43.1s
175:	learn: 3.3727266	total: 17.8s	remaining: 43s
176:	learn: 3.3641764	total: 17.9s	remaining: 42.9s
177:	learn: 3.3638270	total: 18s	remaining: 42.8s
178:	learn: 3.3633774	total: 18.1s	remaining: 42.7s
179:	learn: 3.3633677	total: 18.2s	remaining: 42.4s
180:	learn: 3.3571183	

320:	learn: 2.8278719	total: 33.1s	remaining: 28.8s
321:	learn: 2.8238538	total: 33.3s	remaining: 28.7s
322:	learn: 2.8215959	total: 33.4s	remaining: 28.6s
323:	learn: 2.8178121	total: 33.5s	remaining: 28.5s
324:	learn: 2.8133447	total: 33.6s	remaining: 28.4s
325:	learn: 2.8116193	total: 33.7s	remaining: 28.3s
326:	learn: 2.8111380	total: 33.8s	remaining: 28.2s
327:	learn: 2.8100727	total: 33.9s	remaining: 28.1s
328:	learn: 2.8035540	total: 34s	remaining: 28s
329:	learn: 2.8029638	total: 34.1s	remaining: 27.9s
330:	learn: 2.8018321	total: 34.2s	remaining: 27.8s
331:	learn: 2.8000015	total: 34.3s	remaining: 27.7s
332:	learn: 2.7988697	total: 34.4s	remaining: 27.6s
333:	learn: 2.7978310	total: 34.5s	remaining: 27.5s
334:	learn: 2.7968334	total: 34.6s	remaining: 27.4s
335:	learn: 2.7876109	total: 34.7s	remaining: 27.3s
336:	learn: 2.7844589	total: 34.8s	remaining: 27.2s
337:	learn: 2.7805344	total: 34.9s	remaining: 27.1s
338:	learn: 2.7789395	total: 35s	remaining: 27s
339:	learn: 2.776720

479:	learn: 2.5258142	total: 50s	remaining: 12.5s
480:	learn: 2.5230757	total: 50.1s	remaining: 12.4s
481:	learn: 2.5211483	total: 50.2s	remaining: 12.3s
482:	learn: 2.5207204	total: 50.3s	remaining: 12.2s
483:	learn: 2.5204480	total: 50.4s	remaining: 12.1s
484:	learn: 2.5129959	total: 50.5s	remaining: 12s
485:	learn: 2.5110128	total: 50.6s	remaining: 11.9s
486:	learn: 2.5107505	total: 50.7s	remaining: 11.8s
487:	learn: 2.5104093	total: 50.8s	remaining: 11.7s
488:	learn: 2.5096272	total: 50.9s	remaining: 11.6s
489:	learn: 2.5081127	total: 51s	remaining: 11.4s
490:	learn: 2.5069916	total: 51.1s	remaining: 11.3s
491:	learn: 2.5068525	total: 51.2s	remaining: 11.2s
492:	learn: 2.5054833	total: 51.3s	remaining: 11.1s
493:	learn: 2.5029067	total: 51.4s	remaining: 11s
494:	learn: 2.5017150	total: 51.5s	remaining: 10.9s
495:	learn: 2.4976488	total: 51.6s	remaining: 10.8s
496:	learn: 2.4965473	total: 51.7s	remaining: 10.7s
497:	learn: 2.4949188	total: 51.8s	remaining: 10.6s
498:	learn: 2.492697

In [16]:
print("RMSE:", sqrt(mean_squared_error(y_train, cat.predict(X_train))))

RMSE: 2.364117837241695
